In [1]:
%matplotlib inline
import nivapy3 as nivapy
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import critical_loads as cl
import warnings
import os
import numpy as np
import gdal

warnings.simplefilter('ignore')
plt.style.use('ggplot')

In [2]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis(database='critical_loads')

Connection successful.


# Exceedances of critical loads for water using bias-corrected EMEP data

Kari needs to calculate exceedances of critical loads for water using bias-corrceted EMEP data for the 2020s and 2030s (see e-mail received 23.09.2020 at 08.53 for details). 

## 1. Deposition

The code [here](https://nbviewer.jupyter.org/github/JamesSample/critical_loads_2/blob/master/notebooks/02_deposition_new_grid.ipynb#6.-Change-factors-from-EMEP-data) calculates bias-corrected deposition for the 2030s by first deriving "change factors" between EMEP 2010 and EMEP 2030, and then applying these to BLR-based deposition data for 2007 to 2011. This procedure has been repeated for the 2020s too, and the estimated future deposition datasets are saved e.g. here

    /home/jovyan/shared/critical_loads/raster/deposition/ndep_mgpm2pyr_pred2030_120m.tif

## 2. Exceedances of critical loads for water

Modify the options below to match the decade of interest. 

In [3]:
# Series ID. Use '58bc' for 2020s; '59bc' for 2030s. 
# Note that these are not 'real' series IDs in the db, but can be used here
# as an identifier here nevertheless
series_id = '58bc' 

# Short name used in deposition grid for decade of interest
short_name = "pred2020" # Or 'pred2030'

cell_size = 120
base_path = r"/home/jovyan/shared/critical_loads/raster"

### 2.1. SSWC model

In [4]:
ex_df = cl.calculate_water_exceedance_sswc(series_id, "pred2020", cell_size=120)
ex_df

Exceedance grid saved to:
    /home/jovyan/shared/critical_loads/raster/exceedance/sswc_ex_meqpm2pyr_pred2020_120m.tif


,series_id,medium,total_area_km2,exceeded_area_km2,exceeded_area_pct
0,58bc,water_sswc,320584,14892,5


### FAB model

In [5]:
# Read CL arrays
array_dict = {}
for par in [
    "clminn_meqpm2pyr",
    "clmaxnoaa_meqpm2pyr",
    "clmins_meqpm2pyr",
    "clmaxsoaa_meqpm2pyr",
]:
    # Read tif
    tif_path = os.path.join(base_path, "water", f"{par}_{cell_size}m.tif")
    data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)

    # Set NDV
    data[data == ndv] = np.nan

    # Add to dict
    array_dict[par] = data

# Read dep arrays
for par in ["ndep_mgpm2pyr", "sdep_mgpm2pyr"]:
    # Read tif
    tif_path = os.path.join(
        base_path, "deposition", f"{par}_{short_name}_{cell_size}m.tif"
    )
    data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)
    data = data.astype(np.float32)

    # Set NDV
    data[data == ndv] = np.nan

    # Add to dict
    array_dict[par] = data

In [6]:
%%time
# Extract arrays from dict
cln_min = array_dict["clminn_meqpm2pyr"]
cln_max = array_dict["clmaxnoaa_meqpm2pyr"]
cls_min = array_dict["clmins_meqpm2pyr"]
cls_max = array_dict["clmaxsoaa_meqpm2pyr"]
dep_n = array_dict["ndep_mgpm2pyr"] / 14  # Convert to meq
dep_s = array_dict["sdep_mgpm2pyr"] * 2 / 32.06  # Convert to meq

# Estimate exceedances
ex_n, ex_s, reg_id = cl.vectorised_exceed_ns_icpm(
    cln_min, cln_max, cls_min, cls_max, dep_n, dep_s
)

# Get exceeded area
ex = ex_n + ex_s
ex_area = np.count_nonzero(ex > 0) * cell_size * cell_size / 1.0e6
nor_area = np.count_nonzero(~np.isnan(dep_s)) * cell_size * cell_size / 1.0e6
ex_pct = 100 * ex_area / nor_area

# Build df and tidy
ex_df = pd.DataFrame(
    {
        "exceeded_area_km2": ex_area,
        "total_area_km2": nor_area,
        "exceeded_area_pct": ex_pct,
    },
    index=[0],
)

ex_df = ex_df.round(0).astype(int)
ex_df["series_id"] = series_id
ex_df["medium"] = "water_fab"

ex_df = ex_df[
    ["series_id", "medium", "total_area_km2", "exceeded_area_km2", "exceeded_area_pct"]
]

ex_df

CPU times: user 11 s, sys: 4.98 s, total: 16 s
Wall time: 16 s


,series_id,medium,total_area_km2,exceeded_area_km2,exceeded_area_pct
0,58bc,water_fab,320584,40950,13


In [7]:
# Snap tiff
snap_tif = f"/home/jovyan/shared/critical_loads/raster/blr_land_mask_{cell_size}m.tif"

# N
n_tif = f"/home/jovyan/shared/critical_loads/raster/exceedance/fab_exn_meqpm2pyr_{short_name}_{cell_size}m.tif"
cl.write_geotiff(ex_n, n_tif, snap_tif, -1, gdal.GDT_Float32)

# S
s_tif = f"/home/jovyan/shared/critical_loads/raster/exceedance/fab_exs_meqpm2pyr_{short_name}_{cell_size}m.tif"
cl.write_geotiff(ex_s, s_tif, snap_tif, -1, gdal.GDT_Float32)

# N+S
ns_tif = f"/home/jovyan/shared/critical_loads/raster/exceedance/fab_exns_meqpm2pyr_{short_name}_{cell_size}m.tif"
cl.write_geotiff(ex_n + ex_s, ns_tif, snap_tif, -1, gdal.GDT_Float32)

# Exceedance 'region'
reg_tif = f"/home/jovyan/shared/critical_loads/raster/exceedance/fab_ex_reg_id_{short_name}_{cell_size}m.tif"
cl.write_geotiff(reg_id, reg_tif, snap_tif, -1, gdal.GDT_Float32)